In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd 'drive/My Drive/Colab Notebooks/iub_drive/cam/cifar/dth'

/content/drive/My Drive/Colab Notebooks/iub_drive/cam/cifar/dth


In [4]:
!pip install numpy_indexed

In [5]:
!python process.py

tcmalloc: large alloc 1229201408 bytes == 0x17bf2000 @  0x7f20b6f2f1e7 0x7f20b4a955e1 0x7f20b4af9c78 0x7f20b4af9d93 0x7f20b4b97ea8 0x7f20b4b98704 0x7f20b4b98852 0x566f73 0x59fd0e 0x7f20b4ae54ed 0x50a12f 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x588e5c 0x59fd0e 0x7f20b4ae54ed 0x50a12f 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x50ad03 0x634e72 0x634f27 0x6386df
tcmalloc: large alloc 1475043328 bytes == 0x6faa8000 @  0x7f20b6f2f1e7 0x7f20b4a955e1 0x7f20b4af9c78 0x7f20b4af9d93 0x7f20b4b97ea8 0x7f20b4b98704 0x7f20b4b98852 0x566f73 0x59fd0e 0x7f20b4ae54ed 0x50a12f 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x588e5c 0x59fd0e 0x7f20b4ae54ed 0x50a12f 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x50ad03 0x634e72 0x634f27 0x6386df
tcmalloc: large alloc 1475526656 bytes == 0xc795e000 @  0x7f20b6f2f1e7 0x7f20b4a955e1 0x7f20b4af9c78 0x7f20b4af9d93 0x7f20b4b97ea8 0x7f20b4b98704 0x7f20b4b98852 0x566f73 0x59fd0e 0x7f20b4ae54ed 0x50a12f 0x50beb4 0x507be4 0

In [ ]:
import time
start= time.time()

import torch
from torchvision import datasets
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.functional as F
from torchsummary import summary
from torch.autograd import Variable
from torch.backends import cudnn
from multiprocessing import Pool
import cv2 
from torchvision import transforms, utils
from torch.utils import data
from sklearn.model_selection import train_test_split
import random 
torch.nn.Module.dump_patches = True

import os, glob
import csv

SEED=42
cudnn.benchmark = True
trainL=[0,1,2,3,4,5,6,7]
testL= [8,9]
use_cuda = torch.cuda.is_available()
bs=64
device = torch.device('cuda' if use_cuda else "cpu")
#device = torch.device("cuda:0" if use_cuda else "cpu")
class_total=len(trainL)

random.seed(SEED)
torch.cuda.manual_seed(SEED)

with open('data/all_cifar.npy', 'rb') as f:
    q= np.load(f)

def getData(Q):
    
    for i,val in enumerate(Q):

        if i is 0: loader= q[val, :, :]
        else :
            add=q[val, :, :]
            loader=np.append(loader,add,axis=0)
    
    return loader
 
train=getData(trainL)


x_train, x_test, y_train, y_test = train_test_split(train[:,1:], train[:, 0], test_size=0.2, random_state=42)

np.save("data/novel_test.npy",x_test)
np.save("data/novel_label.npy", y_test)

transform = transforms.Compose([
    transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

class CIFAR(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i, :]
        data = data.astype(np.uint8).reshape(32,32,3)
        
        if self.transforms:
            data = self.transforms(data)
            
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data
 

train_data = CIFAR(x_train, y_train, transform)
trainloader = DataLoader(train_data, batch_size=bs, shuffle=True ,pin_memory=True)

In [ ]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import math


class VGG(nn.Module):

    def __init__(self, features, num_classes=1000):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

def vgg16(STR, **kwargs):
    """VGG 16-layer model (configuration "D")

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = VGG(make_layers(cfg['D']), **kwargs)

    model.load_state_dict(torch.load(STR))
    return model


PATH='vgg16-397923af.pth'

model_vgg = vgg16(PATH)

In [ ]:
model_vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
class custom_vgg (nn.Module):
    def __init__(self):
        super(custom_vgg,self).__init__()
        #self.features = model_vgg.features
        self.features2= model_vgg.features[:15]
        #self.features_vae = model.features
        #self.bn2 =nn.BatchNorm2d(512)
        self.global_avg_pool = nn.AvgPool2d(kernel_size=(7,7))
        self.drop = nn.Dropout(0.7)
        self.fc = nn.Linear(256,class_total)
#         self.soft = nn.Softmax(dim = 1)
        
    def forward(self,x):
        #c1 = self.features(x)
        Cnew=self.features2(x)
        c3 = self.global_avg_pool(Cnew)
        #print(Cnew.shape)
        c4 = c3.view(-1,256)
        
        c5 = self.drop(c4)
        out = self.fc(c5)
#         out = self.soft(c6)
        return Cnew, out # F.relu(Cnew), out  

class VAE(nn.Module):
    def __init__(self,inp,hidden,latent_variable_size):
        super(VAE, self).__init__()

        self.INP=inp
        self.hidden = hidden
        self.latent_variable_size = latent_variable_size
        self.bn = nn.BatchNorm1d(16384)
        self.fc1 = nn.Linear(inp, hidden)
        self.fc21 = nn.Linear(hidden, latent_variable_size)
        self.fc22 = nn.Linear(hidden, latent_variable_size)
        self.fc3 = nn.Linear(latent_variable_size, hidden)
        self.fc4 = nn.Linear(hidden, inp)

    def weight_init(self):
        for block in self._modules:
            try:
                for m in self._modules[block]:
                    normal_init(m)
            except:
                normal_init(block)
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        v=torch.sigmoid(self.fc4(h3))
        return v.reshape(v.size(0),256,8,8)

    def forward(self, x):
        x=x.reshape(x.size(0),-1)
        x=self.bn(x)
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
    
    
def loss_function(recon_x, x, mu, logvar):
    #BCE =F.binary_cross_entropy(recon_x, x, reduction='sum')
    BCE=F.mse_loss(recon_x, x)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    #kld= crit(recon_x, x)
    
    return BCE + KLD ,BCE,KLD


model =custom_vgg()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
vae_loss=[]
epoch_loss=[]
#model_vae = VAE(inp=512*7*7,hidden=5000,latent_variable_size=1000).to(device)
#model_vae = VAE(512*7*7,5000,100)
model_vae = VAE(16384,5000,100)
#model_vae.apply(init_weights)
model_vae.to(device)

optimizer_vae = optim.Adam(model_vae.parameters(), lr=1e-4)
#optimizer_vae = optim.SGD(model_vae.parameters(), lr=1e-3)
max_epochs = 20
model.train()
model_vae.train()

train_loss_min = np.Inf 
for epoch in range(max_epochs):
    train_loss = 0.0
    cn = 0
    epoch_losssss = 0
    acc = 0
    vae_train_loss=0
    bnce=0
    kldl=0
    cl=0
    recl=0
    # Training
    print("Epoch :", epoch+1)

    #itr = (training_set.__len__()) // batch_size
    #print(type(training_generator))
    #print(itr,len(training_generator.dataset))
    
    if __name__ == "__main__":
        for local_batch, local_labels in trainloader:
            cn = cn+1
            # Transfer to GPU
            local_batch, local_labels = local_batch.to(device,non_blocking=True), local_labels.to(device,non_blocking=True)
            #local_batch, local_labels = local_batch.to(device), local_labels.to(device)
            optimizer.zero_grad()


            optimizer_vae.zero_grad()
         
            # forward + backward + optimize
            c1,outputs = model(local_batch)
            
            c1=c1.detach()
            if  torch.isnan(c1).any(): print(torch.isnan(c1).any())
            if  torch.isinf(c1).any(): print(torch.isinf(c1).any())
            #c1=torch.randn(64,256,8,8).to(device)
            recon_batch, mu, logvar = model_vae(c1)
            recon_loss,bce,kld= loss_function(recon_batch,c1 , mu, logvar) #.item()
            #print(VAE_LOSS.item())
            cl_loss=criterion(outputs, local_labels)
            loss_total = cl_loss+recon_loss
            loss,loss_vae= loss_total,loss_total
            
            #vae_loss.append((epoch+1,cn,VAE_LOSS.item(),cl_loss.item(),bce.item(),kld.item()))
            loss.backward(retain_graph=True)
            loss_vae.backward()

            vae_train_loss+=loss_vae.item()*local_batch.size(0)
            kldl+=kld.item()*local_batch.size(0)
            bnce+=bce.item()*local_batch.size(0)
            cl+=cl_loss.item()*local_batch.size(0)
            recl+=recon_loss.item()*local_batch.size(0)

            optimizer.step()
            #torch.nn.utils.clip_grad_norm_(model_vae.parameters(), 5)
            optimizer_vae.step()
    if vae_train_loss <= train_loss_min:
        
        for filename in glob.glob("model_*"):
            os.remove(filename)
        torch.save(model.state_dict(), 'model_vgg_best_epoch'+str(epoch+1)+'.pt')
        torch.save(model_vae.state_dict(), 'model_vae_best_epoch'+str(epoch+1)+'.pt')
        train_loss_min = vae_train_loss

    L=len(trainloader.dataset)
    print(recl,L)
    avg_loss_recl=recl/ L
    avg_loss_cl= cl/L
    avg_loss_kld= kldl/L
    avg_loss_bce=bnce/L
    #print(" vae loss =", avg_loss)
    epoch_loss.append((epoch+1,avg_loss_recl,avg_loss_cl,avg_loss_bce,
                       avg_loss_kld))
    print("epoch {} , recon loss {} , bce loss {} , kld {}",epoch+1,avg_loss_recl,avg_loss_bce, avg_loss_kld)
    #print(" vae loss =", vae_train_loss/ 5)
    #break


with open('cifar_vae_loss_final.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    employee_writer.writerow(["Epoch","classifaction_loss","reconstruction_loss","mse_loss","KLD_loss"])
    for epoch,vloss,clloss,mse,kld in epoch_loss:
        employee_writer.writerow([str(epoch),str(clloss),str(vloss),str(mse),str(kld)])
        
qw=time.time()-start
print(qw/3600)

Epoch : 1
3214055.5247802734 38400
epoch {} , recon loss {} , bce loss {} , kld {} 1 83.69936262448628 3.1320688084761303 80.56729382832845
Epoch : 2
550807.5988464355 38400
epoch {} , recon loss {} , bce loss {} , kld {} 2 14.343947886625926 2.17245763361454 12.171490227381389
Epoch : 3
134233.56314086914 38400
epoch {} , recon loss {} , bce loss {} , kld {} 3 3.495665706793467 2.2269965509573617 1.268669168750445
Epoch : 4
88321.40962219238 38400
epoch {} , recon loss {} , bce loss {} , kld {} 4 2.30003670891126 2.1067908086379368 0.19324590012431145
Epoch : 5
82063.0375289917 38400
epoch {} , recon loss {} , bce loss {} , kld {} 5 2.1370582689841586 2.0735902508099873 0.063468020906051
Epoch : 6
79266.73678588867 38400
epoch {} , recon loss {} , bce loss {} , kld {} 6 2.0642379371325177 2.034882978796959 0.02935496047139168
Epoch : 7
80553.25141906738 38400
epoch {} , recon loss {} , bce loss {} , kld {} 7 2.0977409223715466 2.080593395233154 0.017147526542345682
Epoch : 8
80380.784

In [ ]:
if  torch.isnan(c1).any():
  print(torch.isnan(c1).any())
P= torch.tensor([[1,2,3],[4,5,float('-inf')]])
if  torch.isinf(P).any(): print(torch.isinf(P).any())


tensor(True)


In [ ]:
import pandas as pd
df=pd.read_csv('cifar_vae_loss_final.csv')